# Analiza vegetacije

Satelitski posentki omogočajo pogosto opazovanje velikih površin. Opazujemo lahko stanje in pojave na površju, pri čemer se pogosto usmerjamo na vegetacijo. Staneje vegetacije lahko analiziramo z uporabo podatkov v vidnem in inradrečem delu elektromagnetnega spektra. Satelit Sentinel-2 ima več kanalov, ki so še posebej primerni za spremljanje raslinja.

| Sentinel-2 kanal             | Valovna dolžina (µm) | Ločljivost (m) | Širina kanala (nm) |
|------------------------------|----------------------|----------------|--------------------|
| Band 1 – Coastal aerosol     | 0.443                | 60             | 27/45 (2A/2B)      |
| Band 2 – Blue                | 0.490                | 10             | 98                 |
| Band 3 – Green               | 0.560                | 10             | 45/46 (2A/2B)      |
| Band 4 – Red                 | 0.665                | 10             | 38/39 (2A/2B)      |
| Band 5 – Vegetation Red Edge | 0.705                | 20             | 19/20 (2A/2B)      |
| Band 6 – Vegetation Red Edge | 0.740                | 20             | 18                 |
| Band 7 – Vegetation Red Edge | 0.783                | 20             | 28                 |
| Band 8 – NIR                 | 0.842                | 10             | 115                |
| Band 8A – Narrow NIR         | 0.865                | 20             | 20                 |
| Band 9 – Water vapour        | 0.945                | 60             | 20                 |
| Band 10 – SWIR – Cirrus      | 1.375                | 60             | 20                 |
| Band 11 – SWIR               | 1.610                | 20             | 90                 |
| Band 12 – SWIR               | 2.190                | 20             | 180                |

## Normiran diferencialni vegetacijski indeks NDVI

Ko želimo opisati stanje vegetacije najpogosteje uporabimo normiran diferencialni vegetacijski indeks NDVI. NDVI je bil razvit za opazovanje stanja vegetacije na večjih območjih, na primer na kontinentih ali na celotni Zemlji. NDVI
predstavlja razmerje med razliko infrardečega in rdečega kanala in njuno vsoto:

$$NDVI = \frac{NIR - R}{NIR + R} = \frac{Band_8 - Band_4}{Band_8 + Band_4}$$

Indeks NDVI zavzame vrednosti med −1 in +1, pri čemer višje vrednosti pomenijo bolj intenzivno vegetacijo.

In [ ]:
# Potrebne knjižnice
import numpy as np
import matplotlib.pyplot as plt
import glob
from osgeo import gdal
%matplotlib inline

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
# Velikost grafikonov
plt.rcParams['figure.figsize'] = [12, 8]

## Branje podatkov

Posamezni kanali satelitskih posnetkov Sentinel-2 so v zapisu JPEG2000. Sam format je sicer zelo zapleten, poleg kanalov v različnih ločljivostih vsebuje tudi metapodatke, in je dobro dokumentiran [Sentinel-2 MSI User Guides](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/data-formats).

Format JPEG2000 je podprt v navoejših različicah knjižnice GDAL. Preberemo jih v matriko NumPy.

In [ ]:
# Mapa s podatki
data_folder = "./posnetki_sub/"

In [ ]:
# Seznam posnetkov v mapi
s2_data = glob.glob(data_folder + '*.jp2')

In [ ]:
s2_data

In [ ]:
# Imena datotek
s2_b4_fn = s2_data[0]
s2_b8_fn = s2_data[1]

In [ ]:
# Odpremo datoteki
s2_b4_ds = gdal.Open(s2_b4_fn)
s2_b8_ds = gdal.Open(s2_b8_fn)

## Podatki o posnetku

In [ ]:
s2_b8_ds.GetProjection()

In [ ]:
s2_b8_ds.GetGeoTransform()

In [ ]:
s2_b8_ds.RasterCount

## Preberemo R in IR kanal

In [ ]:
# Kanal 4
s2_b4_band = s2_b4_ds.GetRasterBand(1)
# Preberemo v matriko
s2_b4 = s2_b4_band.ReadAsArray().astype(float)
# Vrednost 0 je nan
s2_b4[s2_b4 == 0] = np.nan

In [ ]:
# Kanal 8
s2_b8_band = s2_b8_ds.GetRasterBand(1)
# Preberemo v matriko
s2_b8 = s2_b8_band.ReadAsArray().astype(float)
# Vrednost 0 je nan
s2_b8[s2_b8 == 0] = np.nan

## Prikažemo oba kanala

In [ ]:
# NIR
plt.imshow(s2_b4)
plt.colorbar()

In [ ]:
# IR
plt.imshow(s2_b8)
plt.colorbar()

In [ ]:
# Histogram kanala 8
plt.hist(s2_b8[~np.isnan(s2_b8)], 50)

In [ ]:
s2_b4_flat = s2_b4.flat
s2_b8_flat = s2_b8.flat

In [ ]:
# Decimiramo
idx = np.random.choice(np.arange(len(s2_b4_flat)), 10000, replace=False)
s2_b4_sample = s2_b4.flat[idx]
s2_b8_sample = s2_b8.flat[idx]
# Razsevni diagram
plt.figure(figsize=(8,8))
plt.scatter(s2_b4_sample, s2_b8_sample, marker=".", s=1)
plt.show()

## Izračunajmo NDVI

Za prenešene posnetke izračunamo NDVI. Upoštevali bomo rdeči (4) in infrardeči kanal (8). Če imamo masko podatkov, lahko upoštevamo tudi manjkajoče podatke. NumPy obvlada deljenje z 0 (vrne ni vrednosti, to je `np.nan`), lahko pa matriki dodamo masko ali pa manjkajoče vrednsoti označimo sami.

In [ ]:
# Definirajmo funkcijo NDVI
def ndvi(red, nir):
    """Calculate NDVI."""
    return (nir - red) / (nir + red)

In [ ]:
# Kanala pretvorimo v float
s2_b4_f = s2_b4.astype(float)
s2_b8_f = s2_b8.astype(float)

In [ ]:
s2_ndvi = ndvi(s2_b4_f, s2_b8)

In [ ]:
# NDVI
plt.imshow(s2_ndvi, cmap='RdYlGn')
plt.colorbar()

In [ ]:
# Histogram
plt.hist(s2_ndvi[~np.isnan(s2_ndvi)], 100)
plt.savefig("test.pdf")

## Shranimo NDVI v datoteko

In [ ]:
# Ime datoteke
s2_ndvi_fn = s2_b4_fn[:-7] + "NDVI.tif"

In [ ]:
driver = gdal.GetDriverByName('GTiff')
ndvi_dataset = driver.Create(s2_ndvi_fn,
                             s2_b4_ds.RasterXSize,    # stolpcev
                             s2_b4_ds.RasterYSize,    # vrstic
                             1,                 # kanalov
                             gdal.GDT_Float32)  # tip podatkov
ndvi_dataset.SetProjection(s2_b4_ds.GetProjection())
ndvi_dataset.SetGeoTransform(s2_b4_ds.GetGeoTransform())

# no data je -1
ndvi_band = ndvi_dataset.GetRasterBand(1)
ndvi_band.SetNoDataValue(-1)

# And finally, let's write our NDVI array.
ndvi_band.WriteArray(s2_ndvi)

## Določitev območij vegetacije

Najprej določimo prag za vegetacijo, nato izdelamo masko samo vegetacije.

In [ ]:
ndvi_tr = 0.2

In [ ]:
s2_veg = np.copy(s2_ndvi)
s2_veg[s2_ndvi >= ndvi_tr] = 1
s2_veg[s2_ndvi < ndvi_tr] = 0

In [ ]:
# Prikaz maske vegetacije
plt.imshow(s2_veg, cmap='RdYlGn')
plt.colorbar()